
# 采用变分算法重构半导体双量子点单-三重态 普适双比特量子门

变分线路构建

采用变强度，不变持续时间的脉冲。

In [6]:
import numpy as np
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/2_qubit_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/2_qubit_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/2_qubit_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/2_qubit_eval_y.npy', allow_pickle=True)
u_mats = np.load('./src/2_qubit_u.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) * dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()
circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)

circ += gate_c_0('0').on([1,0])

circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)
circ += gate_0('010').on(0)
circ += gate_0('011').on(0)

circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)
circ += gate_0('110').on(1)
circ += gate_0('111').on(1)

# circ += gate_c_0('1').on([1,0])

# circ += gate_0('012').on(0)
# circ += gate_1('013').on(0)
# circ += gate_0('014').on(0)
# circ += gate_1('015').on(0)
# circ += gate_0('016').on(0)
# # circ += gate_1('017').on(0)

# circ += gate_0('112').on(1)
# circ += gate_1('113').on(1)
# circ += gate_0('114').on(1)
# circ += gate_1('115').on(1)
# circ += gate_0('116').on(1)
# # circ += gate_1('117').on(1)

circ += gate_c_1('2').on([1,0])

circ += gate_0('018').on(0)
circ += gate_0('019').on(0)
circ += gate_0('020').on(0)
circ += gate_0('021').on(0)
circ += gate_0('022').on(0)
circ += gate_0('023').on(0)

circ += gate_0('118').on(1)
circ += gate_0('119').on(1)
circ += gate_0('120').on(1)
circ += gate_0('121').on(1)
circ += gate_0('122').on(1)
circ += gate_0('123').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)

grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)

acc_list = []
lr = 0.05
for i in range(len(u_mats)):
    params = 2*np.pi* np.random.rand(len(circ.params_name))
    for j in range(len(train_x)):
        f, g = grad_ops(params, train_x[j], train_y[i,j])
        params = abs(params + lr * np.squeeze(g).real)
        
    final_state = []
    for j in range(len(eval_x)):
        sim.reset()
        sim.set_qs(eval_x[j])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
        
    acc = np.real(np.mean([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y[i])]))
    acc_list.append(acc)
    print("\r", end="")
    print(f"进度: {i+1}/{len(u_mats)} ",  'acc:', acc, end="")
    
print('\n平均保真度为：', np.mean(acc_list))

进度: 10/10  acc: 0.978467316260791
平均保真度为： 0.8834858305115736
